In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [3]:
# Load CSV data
df = pd.read_csv("imu_data.csv")

# Drop rows with missing values
df.dropna(inplace=True)

# Features and Labels
X = df[['Roll', 'Pitch']].values
y = df['Direction'].values

In [4]:
# Encode class labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(np.unique(y_encoded))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [10]:
# One-hot encode labels
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(X_train, y_train_cat, epochs=50, batch_size=16, validation_split=0.1)

Epoch 1/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2198 - loss: 32.7083 - val_accuracy: 0.0476 - val_loss: 10.6348
Epoch 2/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2449 - loss: 9.6765 - val_accuracy: 0.1143 - val_loss: 1.8032
Epoch 3/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2529 - loss: 1.6087 - val_accuracy: 0.2190 - val_loss: 1.4380
Epoch 4/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3000 - loss: 1.4143 - val_accuracy: 0.4667 - val_loss: 1.3491
Epoch 5/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4142 - loss: 1.3415 - val_accuracy: 0.5810 - val_loss: 1.2898
Epoch 6/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3631 - loss: 1.3228 - val_accuracy: 0.3048 - val_loss: 1.2880
Epoch 7/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3492 - loss: 1.2827 - val_accuracy: 0.3048 - val_loss: 1.3402
Epoch 8/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4301 - loss: 1.2146 - val_accuracy: 0.4667 - val_los

In [11]:
# Evaluate
loss, acc = model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {acc*100:.2f}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5329 - loss: 1.2701 
Test Accuracy: 53.26%


In [12]:
# Save as TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("imu_model.tflite", "wb") as f:
    f.write(tflite_model)

# Convert to .h file for ESP32
hex_array = ','.join([f'0x{b:02x}' for b in tflite_model])
header_content = f"""#ifndef IMU_MODEL_H
#define IMU_MODEL_H

const unsigned char imu_model_tflite[] = {{
{hex_array}
}};

const int imu_model_tflite_len = {len(tflite_model)};

#endif // IMU_MODEL_H
"""

with open("imu_model.h", "w") as f:
    f.write(header_content)

print("✅ Conversion complete. Files: imu_model.tflite and imu_model.h")

# Save label names for ESP32 mapping
print("Class labels (index -> label):")
for i, label in enumerate(le.classes_):
    print(f"{i}: {label}")

Saved artifact at '/tmp/tmpqojd16ki'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor_12')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  136544582271632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136544582273936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136544582272016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136544582272208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136544582274320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136544582283152: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Conversion complete. Files: imu_model.tflite and imu_model.h
Class labels (index -> label):
0: Down
1: Left
2: Right
3: Up
